In [1]:
%scala
val df = spark.read.load("/FileStore/tables/uz3ks5531497652601093/part_r_00000_8891f92d_5542_4aec_a830_0d4ff6f5f871_gz-87d7c.parquet")
val sales = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")
.coalesce(5)
.where("Description IS NOT NULL")

In [2]:
df = spark.read.load("/FileStore/tables/uz3ks5531497652601093/part_r_00000_8891f92d_5542_4aec_a830_0d4ff6f5f871_gz-87d7c.parquet")
sales = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")\
.coalesce(5)\
.where("Description IS NOT NULL")

In [3]:
%scala
import org.apache.spark.ml.clustering.KMeans
val km = new KMeans().setK(2)
val kmModel = km.fit(df)

In [4]:
from pyspark.ml.clustering import KMeans
km = KMeans().setK(2)
kmModel = km.fit(df)

In [5]:
%scala
val summary = kmModel.summary

In [6]:
summary = kmModel.summary
summary.cluster.show()
summary.clusterSizes

In [7]:
%scala
import org.apache.spark.ml.clustering.BisectingKMeans
val bkm = new BisectingKMeans().setK(2)
val bkmModel = bkm.fit(df)

In [8]:
from pyspark.ml.clustering import BisectingKMeans
bkm = BisectingKMeans().setK(2)
bkmModel = bkm.fit(df)

In [9]:
%scala
val summary = bkmModel.summary

In [10]:
summary = bkmModel.summary
summary.cluster.show()
summary.clusterSizes

In [11]:
%scala
import org.apache.spark.ml.feature.{Tokenizer, CountVectorizer}
val tkn = new Tokenizer()
.setInputCol("Description")
.setOutputCol("DescriptionOut")
val tokenized = tkn.transform(sales)
val cv = new CountVectorizer()
.setInputCol("DescriptionOut")
.setOutputCol("features")
.setVocabSize(500)
.setMinTF(0)
.setMinDF(0)
.setBinary(true)
val prepped = cv.fit(tokenized).transform(tokenized)

In [12]:
from pyspark.ml.feature import Tokenizer, CountVectorizer
tkn = Tokenizer()\
.setInputCol("Description")\
.setOutputCol("DescriptionOut")
tokenized = tkn.transform(sales)
cv = CountVectorizer()\
.setInputCol("DescriptionOut")\
.setOutputCol("features")\
.setVocabSize(500)\
.setMinTF(0)\
.setMinDF(0)\
.setBinary(True)
prepped = cv.fit(tokenized).transform(tokenized)

In [13]:
%scala
import org.apache.spark.ml.clustering.LDA
val lda = new LDA().setK(10).setMaxIter(10)
val model = lda.fit(prepped)

In [14]:
from pyspark.ml.clustering import LDA
lda = LDA().setK(10).setMaxIter(10)
model = lda.fit(prepped)
model.logLikelihood(df)
model.logPerplexity(df)
model.describeTopics(3).show()

In [15]:
%scala
import org.apache.spark.ml.clustering.GaussianMixture
val gmm = new GaussianMixture().setK(2)
val model = gmm.fit(df)
for (i <- 0 until model.getK) {
println(s"Gaussian $i:\nweight=${model.weights(i)}\n" +
s"mu=${model.gaussians(i).mean}\nsigma=\n${model.gaussians(i).cov}\n")
}

In [16]:
from pyspark.ml.clustering import GaussianMixture
gmm = GaussianMixture().setK(2)
model = gmm.fit(df)
for x in model.getK:
  print x

In [17]:
%scala
val summary = model.summary

In [18]:
summary = model.summary
summary.cluster.show()
summary.clusterSizes
summary.probability.show()